In [1]:
import openpyxl
import pyomo.environ as pe
import numpy as np
import os
import pandas as pd

from datetime import datetime
from datetime import timedelta


In [2]:

Year=2019
Mounth=2
Day=14
Hour=13
j=0
T=60


T_ON = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time i
T_OFF = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time 
X_aux = [[0 for j in range(10)] for i in range(T)] # Status of appliance j at time i

# Especifique o diretório onde estão seus arquivos Excel.
diretorio = 'C:/Users/anaol/Documents/NexIK_Flexibility/Data_Valores Reais'

# Lista todos os arquivos na pasta.
arquivos_na_pasta = os.listdir(diretorio)

In [3]:
def function_Ton_T_off (df, j, nome_do_arquivo):

    if nome_do_arquivo in arquivos_na_pasta:
        caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
        
        # Use a função read_excel para carregar os dados em um DataFrame.
        df = pd.read_csv(caminho_arquivo, sep=';')

        # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
    else:
        print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

    df['first_min_on'] = df['first_min_on'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S+00:00"))
    df['last_min_on'] = df['last_min_on'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S+00:00"))
    df['timedelta_on'] = df['timedelta_on'].apply(lambda x: datetime.strptime(x.replace(' ', '').split('days')[1], '%H:%M:%S'))


    for k in range(len(df)):
        if df['first_min_on'][k].second == 30:
            df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
        if df['last_min_on'][k].second == 30:
            df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
        if df['timedelta_on'][k].second == 30:
            df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
        


    for i in range (T):
        for k in range(len(df)):
            if df['first_min_on'][k].year==Year and df['first_min_on'][k].month ==Mounth and df['first_min_on'][k].day == Day and df['first_min_on'][k].hour == Hour:

                if i < df['first_min_on'][k].minute:
                    X_aux[i][j] = 0
                    break
                elif i >=  df['first_min_on'][k].minute and i <=  df['last_min_on'][k].minute and df['last_min_on'][k].hour == Hour:
                    X_aux[i][j] = 1
                    break
                elif i >=  df['first_min_on'][k].minute and df['last_min_on'][k].hour != Hour:
                    X_aux[i][j] = 1
                    break
                else:
                    X_aux[i][j] = 0
                    break
                

In [4]:
# DEVICE DRINKS REFRIGERATOR
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Drinks_Refrigerator.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    

    # Use a função read_excel para carregar os dados em um DataFrame.
    df0 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df0 = df0.iloc[:,0:3]
j=0
function_Ton_T_off (df0, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 0:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [5]:

# DEVICE BLAST CHILLER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Blast_Chiller.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df1 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df1 = df1.iloc[:,0:3]
j=1
function_Ton_T_off (df1, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 1:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')



C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

In [6]:

# DEVICE GARDE MANGER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Garde_Manger_10.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df2 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df2 = df2.iloc[:,0:3]

j=2
function_Ton_T_off (df2, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 2:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 0, 0, 

In [7]:

# DEVICE ICE MACHINE
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Ice_Machine.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df3 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df3 = df3.iloc[:,0:3]
j=3
function_Ton_T_off (df3, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 3:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [8]:

# DEVICE DUAL FRYER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Dual_Fryer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df4 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df4 = df4.iloc[:,0:3]
j=4
function_Ton_T_off (df4, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 4:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [9]:

# DEVICE SALAMANDER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Salamander_1.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df5 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df5 = df5.iloc[:,0:3]
j=5
function_Ton_T_off (df5, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 5:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [10]:

# DEVICE INFRARED SHELF
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Infrared_Shelf.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df6 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df6 = df6.iloc[:,0:3]
j=6
function_Ton_T_off (df6, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 6:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

In [11]:

# DEVICE GLASS WASHER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Glass_Washer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df7 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')


df7 = df7.iloc[:,0:3]
j=7
function_Ton_T_off (df7, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 7:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')



C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [12]:

# DEVICE DISH WASHER
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Dish_Washer.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df8 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df8 = df8.iloc[:,0:3]
j=8
function_Ton_T_off (df8, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 8:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')


i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_min_on'][k] = df['last_min_on'][k] + timedelta(minutes=1)


In [13]:

# DEVICE CONVECTION OVEN
# Escolha o arquivo Excel específico que deseja ler.
nome_do_arquivo = 'activations_Convection_Oven_1.csv'

# Verifique se o arquivo existe na lista de arquivos.
if nome_do_arquivo in arquivos_na_pasta:
    caminho_arquivo = os.path.join(diretorio, nome_do_arquivo)
    
    # Use a função read_excel para carregar os dados em um DataFrame.
    df9 = pd.read_csv(caminho_arquivo, sep=';')

    # Agora, 'df' é o seu DataFrame contendo os dados do Excel.
else:
    print(f'O arquivo {nome_do_arquivo} não foi encontrado na pasta {diretorio}.')

df9 = df9.iloc[:,0:3]
j=9
function_Ton_T_off (df9, j, nome_do_arquivo)

for i in range (T):
    print(f'i: {i}, X: {X_aux[i]}')

#on= [0]*T
#off= [0]*T
cont_on = 0
cont_off = 0
periodo=0
first=0

for i in range (T):
    if j == 9:
        if (i==0 and X_aux[i][j] == 0):
            first = 0
        if X_aux[i][j] == 0:
            cont_off = cont_off + 1
        if X_aux[i][j] == 1:
            cont_on = cont_on + 1
        if X_aux[i][j] == 1 and i<T-1 and (X_aux[i+1][j]==0):
            periodo = i
            for k in range (first, periodo+1):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
            cont_on = 1
            cont_off = 1
            first = periodo+1
        elif X_aux[i][j] == 1 and i==T-1:
            for k in range (first, T):
                T_OFF[k][j] = cont_off 
                T_ON[k][j] = cont_on 
        else:
            if i==T-1:
                for k in range (first, T):
                    if X_aux[i][j] == 1:
                        T_ON[k][j] = cont_on 
                        T_OFF[k][j] = T_ON[k][j]
                    else:
                        T_OFF[k][j] = cont_off -1
                        T_ON[k][j] = T_OFF[k][j]



print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

print('T off')
for i in range (T):
    print(f'i: {i}, on: {T_OFF[i]}')


C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['first_min_on'][k] = df['first_min_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['timedelta_on'][k] = df['timedelta_on'][k] + timedelta(minutes=1)
C:\Users\anaol\AppData\Local\Temp\ipykernel_2068\3625662131.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.

i: 0, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 1, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 2, X: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
i: 3, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 4, X: [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
i: 5, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 6, X: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
i: 7, X: [0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
i: 8, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 9, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 10, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 11, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 12, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 13, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 14, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 15, X: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 16, X: [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
i: 17, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 18, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 19, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 20, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 21, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 22, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 23, X: [1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
i: 24, X: [1, 0, 0, 1, 0, 

In [14]:
for i in range (T):
    print(X_aux[i])

print('T on')
for i in range (T):
    print(f'i: {i}, on: {T_ON[i]}')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 1, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 1]
[1, 0, 0

In [15]:
#################################################################################################################
# Objective: Read some appliance's characteristics from the excel "Info".
#
# ->Dev: Set of devices (1 a 13);
# ->Type: 0 (Not Flexibel), 1 (Power Flexibility), 2 (Time Flexibility) and 3 (Time and Power Flexibility);
# ->T_ON: Time that each load needs to be turned ON;
# ->T_OFF: Time that each load needs to be turned ON and OFF;
# ->State_init: Initial state (t=0) of each appliance: 0 (desligada) ou 1 (ligada);
# ->P_max: Maximum Power for each appliance.
# ->W_max: Maximum Weight for each appliance;
# ->W_min: Minimum Weight for each appliance;
# ->Y_max: Maximum Power percentage for each appliance;
# ->Y_min: Minimum Power percentage for each appliance;
# ->T_Hold_Power: Time needed to fix a power change for each appliance.
#
#################################################################################################################

# load the workbook
workbook = openpyxl.load_workbook('Info.xlsx')

# select the worksheet
worksheet = workbook['Lds_inputs']

# read 'Set of devices'
column = worksheet['B']
Dev = []
for cell in column:
    if cell.row == 1:
        continue
    Dev.append(cell.value)

# read 'Type of appliances'
column = worksheet['C']
Type = []
for cell in column:
    if cell.row == 1:
        continue
    Type.append(cell.value)



# read 'Initial State'
column = worksheet['D']
State_init = []
for cell in column:
    if cell.row == 1:
        continue
    State_init.append(cell.value)

# read 'Maximum Power'
column = worksheet['E']
P_max = []
for cell in column:
    if cell.row == 1:
        continue
    P_max.append(cell.value)

# read 'Maximum Weight'
column = worksheet['F']
W_max = []
for cell in column:
    if cell.row == 1:
        continue
    W_max.append(cell.value)

# read 'Minimum Weight'
column = worksheet['G']
W_min = []
for cell in column:
    if cell.row == 1:
        continue
    W_min.append(cell.value)

# read 'Maximum percentage of power'
column = worksheet['H']
Y_max = []
for cell in column:
    if cell.row == 1:
        continue
    Y_max.append(cell.value)

# read 'Minimum percentage of power'
column = worksheet['I']
Y_min = []
for cell in column:
    if cell.row == 1:
        continue
    Y_min.append(cell.value)

for j in range(len(Dev)):
    Y_max [j] = Y_max [j] / 100
    Y_min [j] = Y_min [j] / 100

# read 'Time needed to fix a power change'
column = worksheet['J']
T_Hold_Power = []
for cell in column:
    if cell.row == 1:
        continue
    T_Hold_Power.append(cell.value)

# close the workbook
workbook.close()

# Print the values
print(f"Index of devices - {(Dev)}")
print(f"Type of appliances - {(Type)}")
#print(f"T_ON - {(T_ON)}")
#print(f"T_OFF - {(T_OFF)}")
print(f"Initial State - {(State_init)}")
print(f"Maximum Power - {(P_max)}")
print(f"Maximum Weight - {(W_max)}")
print(f"Minimum Weight - {(W_min)}")
print(f"Y_max - {(Y_max)}")
print(f"Y_min - {(Y_min)}")
print(f"T_Hold_Power - {(T_Hold_Power)}")


Index of devices - [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Type of appliances - [2, 2, 2, 2, 2, 2, 2, 0, 0, 0]
Initial State - [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Maximum Power - [5000, 1500, 1000, 400, 5000, 3000, 800, 1600, 4200, 7000]
Maximum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Minimum Weight - [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Y_max - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Y_min - [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
T_Hold_Power - [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [16]:

# Set of indices in scheduling horizon        
P_lim = 12000 # Power limit      
rows = T # Auxiliary variable
cols = len(Dev) # Auxiliary variable 


# On/Off variables:
X_onoff_RT_vector = [0] * len(Dev) # Status of appliance j
X_onoff_RT_matrix = [[0 for j in range(cols)] for i in range(rows)] # Status of appliance j at time i
X_onoff_opt_RT_vector = [0] * len(Dev) # Optimized Status of appliance j


# Power change variables:
Y_RT_matrix = [[1 for j in range(cols)] for i in range(rows+1)] # Percentage of Power for each appliance j at time i 
Y_opt_RT_vector = [1] * len(Dev) # Percentage of Optimized Power for each appliance j


# Power variables:
P_RT = [0] * len(Dev) # Power of appliance j
P_sum=[0] * T # Sum of Power of all appliances at time i
P_opt_RT = [0] * len(Dev) # Optimized Power for each appliance j
P_opt_sum=[0] * T # Sum of Optimized Power of all appliances at time i
P_RT_matrix = [[0 for j in range(cols)] for i in range(rows)]

# Weight variables:
W_RT_vector = [10] * len(Dev) # Weight of appliance j
W_RT_matrix = [[10 for j in range(cols)] for i in range(rows)] # Weight of appliance j at time i
W_opt_RT_vector = [10] * len(Dev) # Optimized Weight of appliance j
weight_vector_before_optim = [['' for j in range(cols)] for i in range(rows)]
W_Power = [[10 for j in range(cols)] for i in range(rows)] 

# Auxiliary variables:
Optimization =0
X_onoff_before_opt_RT = [['' for j in range(cols)] for i in range(rows)]
y_before_opt = [['' for j in range(cols)] for i in range(rows)]
cont_on_dataset = [0] * len(Dev)
cont_off_dataset = [0] * len(Dev)
count_T_Hold_Power = [0] * len(Dev)
count_T_Hold_Power_matrix = [[0 for j in range(cols)] for i in range(rows)]
cont_on_weight = [0] * len(Dev)
cont_off_weight = [0] * len(Dev)
W_Power_aux = [0] * len(Dev)
P_rlx_matrix = [0] * T



T_ON_atualizado = [0] * len(Dev)
T_OFF_atualizado = [0] * len(Dev)


In [17]:

def function_initialize(X_onoff_RT_vector, W_RT_vector, P_max):
    
    for j in range(len(Dev)): 
        P_RT[j] = P_max[j]        
        # Loads with time flexibility
        if ((Type[j]==2 or Type[j]==3)):
            if (X_onoff_RT_vector[j] == 1): # Start ON                         # State ON
                W_RT_vector[j] = 10 - (9/T_ON_atualizado[j])   # Weight
                
            else: # Start OFF                     # State OFF
                W_RT_vector[j] = 1 + (9/T_OFF_atualizado[j])              # Weight

        # Loads non-flexible
        # Loads with power flexibility
        if Type[j] == 0 or Type[j] == 1: 
                                                 
            W_RT_vector[j] = 10                                # Weight
    
    return W_RT_vector, P_RT

               

In [18]:

def function_dataset(i,X_onoff_RT_matrix,W_RT_matrix, X_aux): #X_onoff_RT_matrix, 
    
    for j in range(len(Dev)):

        # Loads with time flexibility
        if (Type[j]==2 or Type[j]==3):

            if i != 0 and X_onoff_RT_matrix[i-1][j] == 1: # State ON, from Weight=10 to Weight=1
                if W_RT_matrix[i-1][j] <= 1:                    
                    X_onoff_RT_vector[j] = 0 # Turn off the load

            elif i !=0 and X_onoff_RT_matrix[i-1][j] == 0: # State OFF, from Weight=1 to Weight=10              
                if W_RT_matrix[i-1][j] >= 10:                    
                    X_onoff_RT_vector[j] = 1 # Turn on the load

        if (Type[j]==0 or Type[j]==1):
            X_onoff_RT_vector[j]= X_aux[i][j]


        
    return (X_onoff_RT_vector)
    

In [19]:

def function_weight (i, Update_variables, X_onoff_RT_vector, X_onoff_opt_RT_vector, W_RT_matrix):
   
    if Update_variables == 0: # Before Optimization: 
       
        for j in range(len(Dev)): 

            # Loads with time flexibility
            if (Type[j]==2 or Type[j]==3):
                if W_RT_matrix[i-1][j] > 10: # If the Weight before is higher than 10, we have to force the device to turn on
                    W_RT_vector[j] = -1

                else:
                    if X_onoff_RT_vector[j] == 1 : # State ON                                                                                               
                        W_RT_vector[j] =  W_RT_matrix [i-1][j] - (9/T_ON_atualizado[j] ) # Weight decreases
                    if X_onoff_RT_vector[j] == 0 : # State OFF 
                        W_RT_vector[j] =  W_RT_matrix [i-1][j] + (9/T_OFF_atualizado[j] ) # Weight increases
      
           # W_RT_vector[j]=round(W_RT_vector[j], 5)
        return (W_RT_vector)


    elif Update_variables == 1: # After Optimization: 
       
        for j in range(len(Dev)): 
            
            # Loads with time flexibility
            # If the optimization turned off the load
            if (Type[j]==2 or Type[j]==3) and (X_onoff_RT_vector[j]==1 and X_onoff_opt_RT_vector[j])==0:  
                if  i!=0 and W_RT_matrix[i-1][j]!=-1: 
                    W_RT_vector[j] = W_RT_matrix[i-1][j] + 9/T_OFF_atualizado[j] # Weight increases

            # Loads non-flexible
            # Loads with power flexibility
            if Type[j] == 0 or Type[j] == 1:
                W_RT_vector[j] = 10
            
        return (W_RT_vector)
    

    elif Update_variables == 2: # End of Event: 
        for j in range(len(Dev)):
            if (Type[j]==2 or Type[j]==3) and W_RT_vector[j] == -1:
                W_RT_vector[j] = W_RT_matrix[i-1][j] - 9/T_ON_atualizado[j]

        return (W_RT_vector)



             

In [20]:

def function_P_sum (X_onoff_RT_vector, P_RT):
    
    P_sum = 0
    
    for j in range(len(Dev)):     
        if X_onoff_RT_vector[j] == 1: # State ON
            P_sum = P_sum + P_RT[j]
    
    return(P_sum)


In [21]:

def function_power (i, Update_variables, X_onoff_RT_matrix, count_T_Hold_Power_matrix):

    if Update_variables==0:

        # Y, Power
        for j in range(len(Dev)):
            if (Type[j] == 1 or Type[j] == 3):
                if X_onoff_RT_matrix[i-1][j]== 1:   
                    if count_T_Hold_Power_matrix[i-1][j] != 0 and count_T_Hold_Power_matrix[i-1][j] < T_Hold_Power[j]:
                        count_T_Hold_Power[j] = count_T_Hold_Power_matrix[i-1][j] +1
                        Y_opt_RT_vector[j] = Y_RT_matrix[i-1][j]
                        
                    elif count_T_Hold_Power_matrix[i-1][j] == T_Hold_Power[j]:
                        count_T_Hold_Power[j] = 0
                        Y_opt_RT_vector[j] = 1

                    else:
                        Y_opt_RT_vector[j] = 1
                        count_T_Hold_Power[j] = 0

                else:
                    Y_opt_RT_vector[j] = 1
                    count_T_Hold_Power[j] = 0
                   
        
            #Vector Power and T_ON Update
            P_RT[j] = P_max [j]*Y_opt_RT_vector[j]*X_onoff_RT_matrix[i][j]
            
            if Type[j] == 3: #  Loads with time and power flexibility
                T_ON_atualizado[j] = (P_max[j]*T_ON[j]) / P_RT[j]
        
       

        return(Y_opt_RT_vector, count_T_Hold_Power, P_RT)
    




    elif Update_variables == 2:
        
        for j in range(len(Dev)):
            if Type[j] == 1 or Type[j] == 3:
                if i!=0 and Y_RT_matrix[i][j] != Y_RT_matrix[i-1][j] and X_onoff_RT_matrix[i][j]==1 and count_T_Hold_Power_matrix [i][j]==0:  
                    count_T_Hold_Power [j] = 1
                  

                if i==0 and X_onoff_RT_matrix[i][j]==1 and count_T_Hold_Power_matrix [i][j]==0:  
                    count_T_Hold_Power [j] = 1
                  
                    

        return(count_T_Hold_Power)


In [22]:

model = pe.ConcreteModel()

# Set Declaration
model.loads = pe.Set(initialize=np.arange(len(Dev)))
#model.loads = pe.Set(initialize=range(len(Dev)))

# Parameter Declaration
model.lim = pe.Param(initialize=P_lim)
model.P_max = pe.Param(range(len(Dev)),initialize=P_max)
model.T_Hold_Power = pe.Param(range(len(Dev)),initialize=T_Hold_Power)
model.W_time =pe.Param(range(len(Dev)),initialize=0, mutable=True)
model.W_max = pe.Param(range(len(Dev)),initialize=W_max)
model.W_min = pe.Param(range(len(Dev)),initialize=W_min)
model.Y_max = pe.Param(range(len(Dev)),initialize=Y_max)
model.Y_min = pe.Param(range(len(Dev)),initialize=Y_min)
model.X =pe.Param(range(len(Dev)),initialize=0, mutable=True)
model.Y =pe.Param(range(len(Dev)),initialize=1, mutable=True)
model.count_T_Hold_Power =pe.Param(range(len(Dev)),initialize=0, mutable=True)

# Variable Declaration
model.X_opt = pe.Var(range(len(Dev)),domain=pe.Binary, initialize={i: 1 for i in range(len(Dev))})
model.Y_opt = pe.Var(range(len(Dev)),domain=pe.NonNegativeReals, initialize={i: 1 for i in range(len(Dev))})
model.P_rlx = pe.Var(domain=pe.NonNegativeReals, initialize=0)

# Constrait 1
def _constraint_one(m):
   return sum((m.P_max[j]*m.X_opt[j]*m.Y_opt[j]) for j in range(len(Dev))) <= m.lim + m.P_rlx
model.constraint_one = pe.Constraint(rule=_constraint_one)

# Constrait 2
def _constraint_two(m, j):
        return ((m.Y_min[j] <= m.Y_opt[j]))
model.constraint_two = pe.Constraint(model.loads, rule=_constraint_two)

# Constrait 3
def _constraint_three(m, j):
    return ((m.Y_opt[j] <= m.Y_max[j]))
model.constraint_three = pe.Constraint(model.loads, rule=_constraint_three)

# Objective function
def _objective_function(m):
    return ( sum(m.W_time[j]*m.X_opt[j]+ ((m.W_min[j] + ((m.W_max[j] - m.W_min[j]) / (m.Y_max[j] - m.Y_min[j] + 0.0001)) * (m.Y_opt[j] - m.Y_min[j]))*m.X_opt[j]) for j in range(len(Dev))) ) - (10000*m.P_rlx)
model.objective = pe.Objective(expr=_objective_function, sense=pe.maximize)


In [23]:

for i in range (T): 
    for j in range(len(Dev)):
        T_ON_atualizado[j] = T_ON[i][j]
        T_OFF_atualizado[j] =T_OFF[i][j]


    # Before Optimization
    Update_variables=0 
    
    if i==0: # Vectors Initialization
       
        for j in range(len(Dev)):
            X_onoff_RT_matrix[i][j] = X_aux[i][j]



        X_onoff_RT_vector = X_onoff_RT_matrix[i]
        W_RT_vector, P_RT = function_initialize(X_onoff_RT_vector, W_RT_vector, P_max)
        #X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy() # Status on/off of loads
        W_RT_matrix [i] = W_RT_vector.copy() # Weight of loads

    else: 
        # Status on/off of loads
        X_onoff_RT_vector = function_dataset (i, X_onoff_RT_matrix, W_RT_matrix, X_aux)     #X_onoff_RT_matrix,                           
        X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy()
        
        # Weight of loads
        W_RT_vector = function_weight(i, Update_variables, X_onoff_RT_vector, X_onoff_opt_RT_vector, W_RT_matrix) 
        W_RT_matrix [i] = W_RT_vector.copy()

        # Power of loads   
        
        Y_opt_RT_vector, count_T_Hold_Power, P_RT = function_power(i, Update_variables, X_onoff_RT_matrix, count_T_Hold_Power_matrix)
        Y_RT_matrix[i] = Y_opt_RT_vector.copy()
        count_T_Hold_Power_matrix [i] = count_T_Hold_Power.copy()
        P_RT_matrix[i] = P_RT.copy()
        

    # Sum of load's power
    P_sum[i] = function_P_sum(X_onoff_RT_vector, P_RT)                                             

    if P_sum[i] > P_lim: # Check the need for optimization
        
        # Initialization of some optimization parameters
        for j in range(len(Dev)):                                                       
            model.W_time[j] = W_RT_vector[j]  # Time Component of Weight
            model.X[j] = X_onoff_RT_vector[j] # Status on/off of each loads
            model.Y[j] = Y_opt_RT_vector[j]   # Percentage of each load's power
            model.count_T_Hold_Power[j] = count_T_Hold_Power[j] # Counter of T_Hold

        # Delete existing components in constraint 4 before adding new components with the same name.
        if hasattr(model, 'constraint_four'):
            model.del_component(model.constraint_four)

        # Delete existing components in constraint 5 before adding new components with the same name.
        if hasattr(model, 'constraint_five'):
            model.del_component(model.constraint_five)

        # Delete existing components in constraint 6 before adding new components with the same name.
        if hasattr(model, 'constraint_six'):
            model.del_component(model.constraint_six)

        # Constraint 4
        def _constraint_four(m,j):  
            # Loads non-flexible
            # Loads with power flexibility
            if Type[j] == 0 or Type[j] == 1 or W_RT_vector[j] == -1: 
                return m.X_opt[j] == m.X[j] # The optimization status on/off must be the same as the before optimization
            # Loads with time flexibility
            if (Type[j] == 2 or Type[j] == 3) and W_RT_vector[j] != -1:
                return m.X_opt[j] <= m.X[j] # The optimization can keep or turn off the load's status
            else:
                return (pe.Constraint.Feasible)
        model.constraint_four = pe.Constraint(model.loads, rule=_constraint_four)

        # Constraint 5
        def _constraint_five(m): # DishWasher
            # Change of power 
            if X_onoff_RT_vector[7]==1 and X_onoff_RT_matrix[i-1][7]==1:
                return (m.X_opt[8] ==  m.X[8]) # The optimization percentage of each load's power must be the same as the before optimization 
            else:
                return (pe.Constraint.Feasible) 
        model.constraint_five = pe.Constraint(rule=_constraint_five)

        # Constraint 5
        def _constraint_five(m): # DishWasher
            # Change of power 
            if X_onoff_RT_vector[8] == 1 and X_onoff_RT_matrix[i-1][8]==1:
                return (m.X_opt[8] ==  m.X[8]) # The optimization percentage of each load's power must be the same as the before optimization 
            else:
                return (pe.Constraint.Feasible) 
        model.constraint_five = pe.Constraint(rule=_constraint_five)

        # Constraint 6 
        def _constraint_six(m,j): 
            # Change of power 
            if count_T_Hold_Power[j]!=0: #and cont_T_Hold_Power[j] != m.T_Hold_Power [j]: 
                return (m.Y_opt[j] ==  m.Y[j]) # The optimization percentage of each load's power must be the same as the before optimization 
            # Power is maintained until T_Hold
            else:
                return (pe.Constraint.Feasible) # The optimization change the percentage of power
        model.constraint_six = pe.Constraint(model.loads, rule=_constraint_six)

        # Solver                                                 
        results = pe.SolverFactory('scip', executable='C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe').solve(model)
        #model.pprint()
        
        # Save the results before optimization
        X_onoff_before_opt_RT[i] = X_onoff_RT_vector.copy()
        weight_vector_before_optim[i] = W_RT_vector.copy()
        y_before_opt[i] = Y_opt_RT_vector.copy()

        # After Optimization
        Update_variables=1 

        # Save the optimization results
        for j in range(len(Dev)):
            X_onoff_opt_RT_vector [j] = int (np.abs(model.X_opt[j].value)) # Status on/off of each loads after optimization
            if X_onoff_opt_RT_vector [j] == 1:
                Y_opt_RT_vector [j] = (model.Y_opt[j].value) # Percentage of each load's power after optimization
                P_RT[j] = (model.P_max[j]*model.X_opt[j].value*model.Y_opt[j].value) # Power of each load after optimization
            else:
                P_RT[j] = 0
            # Power component of weight
            W_Power_aux [j] = ((W_min[j] + ((W_max[j] - W_min[j]) / (Y_max[j] - Y_min[j] + 0.0001)) * (Y_opt_RT_vector [j] - Y_min[j])))

        W_Power[i]= W_Power_aux.copy() # Power component of weight
        P_rlx_matrix[i] = model.P_rlx.value # Power that exceeds the limit
        Objective_Function_RT = model.objective() # Result of objective function

        # Update the status on/off of loads
        X_onoff_RT_vector = X_onoff_opt_RT_vector.copy()
        X_onoff_RT_matrix[i] = X_onoff_RT_vector.copy() 

        # Update the weight of loads                                                 
        W_RT_vector = function_weight(i, Update_variables, X_onoff_before_opt_RT[i], X_onoff_opt_RT_vector, W_RT_matrix)
        W_RT_matrix [i] = W_RT_vector.copy()

        # Update the power of loads
        Y_RT_matrix[i]= Y_opt_RT_vector.copy() # Percentage of each load's power after optimization
        P_RT_matrix[i] = P_RT.copy() # Power of each load after optimization

        # Update the sum of power
        P_opt_sum[i] = function_P_sum(X_onoff_RT_vector, P_RT)                                     


    Update_variables=2

    # Update the weight of loads
    W_RT_vector = function_weight(i, Update_variables, X_onoff_before_opt_RT[i], X_onoff_opt_RT_vector, W_RT_matrix)
    W_RT_matrix [i] = W_RT_vector.copy()

    # Update the counter T hold
    count_T_Hold_Power = function_power(i, Update_variables, X_onoff_RT_matrix, count_T_Hold_Power_matrix)
    count_T_Hold_Power_matrix [i] = count_T_Hold_Power.copy()

    # Print of Results
    print(f"\n################################################# Periodo {i} #################################################\n")
    print(f"Optimization = {Optimization}\n")
    print(f"Variables related to TIME:")
    print(f"X before optimization = {(X_onoff_before_opt_RT[i])}") # If there is optimization
    print(f"X Final = {(X_onoff_opt_RT_vector)}")
    print(f"Weight Time before optimization = {(weight_vector_before_optim[i])}")  
    print(f"Weight Time Final = {(W_RT_vector)}\n")  

    print(f"Variables related to POWER:")
    print(f"Weight Power = {(W_Power_aux)}")
    print(f"Power Max = {(P_max)}")
    print(f"Power Final = {(P_RT_matrix[i])}")
    print(f"y = {(Y_opt_RT_vector)}")
    print(f"Counter T Hold= {(count_T_Hold_Power)}")
    print(f"T Hold= {(T_Hold_Power)}")
    print(f"Power Rlx = {(P_rlx_matrix [i])}\n")

    print(f"Objective Funtion")
    print(f"Maximizar OF = {model.objective()}")


################################################# Periodo 0 #################################################

Optimization = 0

Variables related to TIME:
X before optimization = ['', '', '', '', '', '', '', '', '', '']
X Final = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Weight Time before optimization = ['', '', '', '', '', '', '', '', '', '']
Weight Time Final = [1.5625, 1.183673469387755, 4.0, 2.2857142857142856, 4.0, 1.1956521739130435, 1.5294117647058822, 10, 10, 10]

Variables related to POWER:
Weight Power = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Power Max = [5000, 1500, 1000, 400, 5000, 3000, 800, 1600, 4200, 7000]
Power Final = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
y = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Counter T Hold= [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
T Hold= [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
Power Rlx = 0

Objective Funtion
Maximizar OF = 100

################################################# Periodo 1 #################################################

Optimization = 0

Variables related to TIME:
X

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "scip"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: C:/Program Files/SCIPOptSuite 8.0.3/bin/scip.exe
	type: scip
	_args: ()
	options: {}

In [ ]:


import datetime

col = []
for i in range(T):
    # Crie um objeto datetime representando o tempo
    time_value = datetime.datetime(1, 1, 1, i // 60, i % 60)

    # Formate o objeto datetime como uma string no formato desejado (hora:minuto:segundo)
    time_str = time_value.strftime("%H:%M:%S")

    col.append(time_str)




for i in range(T):
    X_onoff_RT_matrix[i].insert(0, col[i])
    X_onoff_before_opt_RT[i].insert(0, col[i])
    W_RT_matrix[i].insert(0, col[i])
    weight_vector_before_optim[i].insert(0, col[i])
    count_T_Hold_Power_matrix[i].insert(0, col[i])
    Y_RT_matrix[i].insert(0, col[i])
    y_before_opt[i].insert(0, col[i])
    P_RT_matrix[i].insert(0, col[i])
    W_Power[i].insert(0, col[i])
    T_ON[i].insert(0, col[i])
    T_OFF[i].insert(0, col[i])

for i in range(T):
    X_onoff_RT_matrix[i].append(0)
    X_onoff_RT_matrix[i].append(0)
    X_onoff_RT_matrix[i].append(0)

    P_RT_matrix[i].append(0)
    P_RT_matrix[i].append(0)
    P_RT_matrix[i].append(0)

    #change value in ON_or_OFF list in position 16 for sum[i]
    X_onoff_RT_matrix[i][11] = P_sum[i]
    X_onoff_RT_matrix[i][12] = P_opt_sum[i]
    X_onoff_RT_matrix[i][13] = P_rlx_matrix[i]

    P_RT_matrix[i][11] = P_sum[i]
    P_RT_matrix[i][12] = P_opt_sum[i]
    P_RT_matrix[i][13] = P_rlx_matrix[i]
    
row = []
for i in range(len(Dev)+4):
    row.append(i)
row [0] = 'H:M:S / Index'
row [11] ='Sum Power'
row [12] ='Power Optim'
row [13] ='Power Rlx'
row1 = []
for i in range(len(Dev)+1):
    row1.append(i)
row1 [0] = 'H:M:S / Index'

X_onoff_RT_matrix.insert(0, row)
X_onoff_before_opt_RT.insert(0, row1)
W_RT_matrix.insert(0, row1)
weight_vector_before_optim.insert(0, row1)
count_T_Hold_Power_matrix.insert(0, row1)
Y_RT_matrix.insert(0, row1)
y_before_opt.insert(0, row1)
P_RT_matrix.insert(0, row)
W_Power.insert(0, row1)
T_ON.insert(0, row1)
T_OFF.insert(0, row1)


In [ ]:

# Create a Pandas Excel writer using XlsxWriter
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')

df = pd.DataFrame(X_onoff_RT_matrix)
writer = pd.ExcelWriter('Results.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='X', index=False, header=False)

df2 = pd.DataFrame(X_onoff_before_opt_RT)
df2.to_excel(writer, sheet_name='X before opt', index=False, header=False)

df3 = pd.DataFrame(W_RT_matrix)
df3.to_excel(writer, sheet_name='W_Time', index=False, header=False)

df4 = pd.DataFrame(weight_vector_before_optim)
df4.to_excel(writer, sheet_name='W_Time before opt', index=False, header=False)

df5 = pd.DataFrame(count_T_Hold_Power_matrix)
df5.to_excel(writer, sheet_name='counter_change power', index=False, header=False)

df6 = pd.DataFrame(Y_RT_matrix)
df6.to_excel(writer, sheet_name='Y', index=False, header=False)

df7 = pd.DataFrame(y_before_opt)
df7.to_excel(writer, sheet_name='Y before opt', index=False, header=False)

df8 = pd.DataFrame(P_RT_matrix)
df8.to_excel(writer, sheet_name='Power(j)', index=False, header=False)

df9 = pd.DataFrame(W_Power)
df9.to_excel(writer, sheet_name='W_Power(j)', index=False, header=False)

df10 = pd.DataFrame(T_ON)
df10.to_excel(writer, sheet_name='Ton', index=False, header=False)

df9 = pd.DataFrame(T_OFF)
df9.to_excel(writer, sheet_name='Toff', index=False, header=False)


writer.save()
writer.close()


C:\Users\anaol\AppData\Local\Temp\ipykernel_18176\1776833869.py:49: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
c:\Users\anaol\AppData\Local\Programs\Python\Python311\Lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
